# FINAL PROJECT - SOCIAL MEDIA - AML3204

##1.  IMPORTING LIBRARIES

In [1]:
## Basic libraries
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
import datetime
## Libraries for the models
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

## 2. READING DATA

### 2.1  Dataset for Ratings

In [2]:
# Reading data to get Ratings
head = ["user_id","item_id","rating","timestamp"]
df_rating = pd.read_csv("sample_data/u.data",names=head,sep="\t")

In [3]:
df_rating

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [4]:
np.sort( df_rating.user_id.unique() )

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [5]:
np.sort( df_rating.item_id.unique() )

array([   1,    2,    3, ..., 1680, 1681, 1682])

In [6]:
np.sort( df_rating.rating.unique() )

array([1, 2, 3, 4, 5])

In [7]:
df_rating["datetime"] = df_rating["timestamp"].apply(lambda x: datetime.datetime.fromtimestamp(x))

In [8]:
df_pivoted = df_rating.pivot( index="user_id" , columns="item_id" , values="rating" )
df_pivoted

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_rating.loc[ df_rating["user_id"] == 1 ].sort_values(by = "item_id").head(10)

,user_id,item_id,rating,timestamp,datetime
32236,1,1,5,874965758,1997-09-22 22:02:38
23171,1,2,3,876893171,1997-10-15 05:26:11
83307,1,3,4,878542960,1997-11-03 07:42:40
62631,1,4,3,876893119,1997-10-15 05:25:19
47638,1,5,3,889751712,1998-03-13 01:15:12
5533,1,6,5,887431973,1998-02-14 04:52:53
70539,1,7,4,875071561,1997-09-24 03:26:01
31650,1,8,1,875072484,1997-09-24 03:41:24
20175,1,9,5,878543541,1997-11-03 07:52:21
13542,1,10,3,875693118,1997-10-01 08:05:18


In [10]:
df_rating.loc[ df_rating["user_id"] == 940 ].sort_values(by = "item_id").head(10)

,user_id,item_id,rating,timestamp,datetime
72987,940,4,2,885922040,1998-01-27 17:27:20
97515,940,7,4,885921597,1998-01-27 17:19:57
69670,940,8,5,885921577,1998-01-27 17:19:37
78057,940,9,3,885921687,1998-01-27 17:21:27
89279,940,12,4,885921979,1998-01-27 17:26:19
67014,940,14,3,885921710,1998-01-27 17:21:50
84316,940,47,3,885921758,1998-01-27 17:22:38
87299,940,50,4,885921542,1998-01-27 17:19:02
70035,940,56,5,885921577,1998-01-27 17:19:37
67362,940,66,4,885922016,1998-01-27 17:26:56


In [11]:
df_rating.isnull().sum()

user_id      0
item_id      0
rating       0
timestamp    0
datetime     0
dtype: int64

### 2.2 Dataset for Movies

In [12]:
columns = ["item_id" , "movie_title" , "release_date" , "video_release_date" , "IMDb_URL" , "unknown" , "Action" ,
           "Adventure" , "Animation" , "Children's" , "Comedy" , "Crime" , "Documentary" , "Drama" , "Fantasy" ,
           "Film-Noir" , "Horror" , "Musical" , "Mystery" , "Romance" , "Sci-Fi" , "Thriller" , "War" , "Western"]
df_movies = pd.read_csv( "sample_data/u.item" , names = columns , encoding = "cp1252" , delimiter = "|" )

In [13]:
df_movies

,item_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
print( ' The dimensions of movies dataframe are : ' , df_movies.shape )

 The dimensions of movies dataframe are :  (1682, 24)


In [15]:
print( ' The dimensions of ratings dataframe are : ' , df_rating.shape )

 The dimensions of ratings dataframe are :  (100000, 5)


## 3. ANALYSING THE SPARSE MATRIX

In [16]:
# Movie ID to movie name mapping
movie_names = df_movies.set_index('item_id')['movie_title'].to_dict()

In [17]:
movie_names

{1: 'Toy Story (1995)',
 2: 'GoldenEye (1995)',
 3: 'Four Rooms (1995)',
 4: 'Get Shorty (1995)',
 5: 'Copycat (1995)',
 6: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 7: 'Twelve Monkeys (1995)',
 8: 'Babe (1995)',
 9: 'Dead Man Walking (1995)',
 10: 'Richard III (1995)',
 11: 'Seven (Se7en) (1995)',
 12: 'Usual Suspects, The (1995)',
 13: 'Mighty Aphrodite (1995)',
 14: 'Postino, Il (1994)',
 15: "Mr. Holland's Opus (1995)",
 16: 'French Twist (Gazon maudit) (1995)',
 17: 'From Dusk Till Dawn (1996)',
 18: 'White Balloon, The (1995)',
 19: "Antonia's Line (1995)",
 20: 'Angels and Insects (1995)',
 21: 'Muppet Treasure Island (1996)',
 22: 'Braveheart (1995)',
 23: 'Taxi Driver (1976)',
 24: 'Rumble in the Bronx (1995)',
 25: 'Birdcage, The (1996)',
 26: 'Brothers McMullen, The (1995)',
 27: 'Bad Boys (1995)',
 28: 'Apollo 13 (1995)',
 29: 'Batman Forever (1995)',
 30: 'Belle de jour (1967)',
 31: 'Crimson Tide (1995)',
 32: 'Crumb (1994)',
 33: 'Desperado (1995)',
 34: '

In [18]:
# Getting the number of users and items from df_rating
n_users = len(df_rating.user_id.unique())
n_items = len(df_rating.item_id.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(df_rating))
print("Therefore: ", len( df_rating ) / ( n_users * n_items ) * 100, '% of the matrix is filled.')
### Analizing the sparse matrix
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 943
Number of unique movies: 1682
The full rating matrix will have: 1586126 elements.
----------
Number of ratings: 100000
Therefore:  6.304669364224531 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


## 4. Defining the model to calculate the Matrix Factorization

In [19]:
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=8):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.

        # Filling self tensor with numbers sampled from the continuous uniform distribution
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        #print(users)
        return (self.user_factors(users)*self.item_factors(items)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

## 5. Getting the Data from DF rating

In [20]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

In [21]:
# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = df_rating.copy()

        # Extract all user IDs and movie IDs
        users  = df_rating.user_id.unique()
        movies = df_rating.item_id.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.item_id = df_rating.item_id.apply(lambda x: self.movieid2idx[x])
        self.ratings.user_id  = df_rating.user_id.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp' , 'datetime'], axis=1).values
        self.y = self.ratings['rating'].values

        ## Transforming the data to tensors (ready for torch models.)
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

## 6. INITIALIZING THE MATRIX FACTORIZATION

In [22]:
cuda = torch.cuda.is_available()
print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print( name , "---" , param.data )

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(943, 8)
  (item_factors): Embedding(1682, 8)
)
user_factors.weight --- tensor([[0.0300, 0.0118, 0.0327,  ..., 0.0450, 0.0269, 0.0428],
        [0.0203, 0.0094, 0.0205,  ..., 0.0466, 0.0499, 0.0114],
        [0.0277, 0.0230, 0.0412,  ..., 0.0149, 0.0271, 0.0274],
        ...,
        [0.0109, 0.0040, 0.0077,  ..., 0.0145, 0.0079, 0.0206],
        [0.0013, 0.0452, 0.0347,  ..., 0.0137, 0.0358, 0.0013],
        [0.0415, 0.0209, 0.0368,  ..., 0.0127, 0.0183, 0.0347]])
item_factors.weight --- tensor([[0.0245, 0.0425, 0.0218,  ..., 0.0280, 0.0400, 0.0259],
        [0.0104, 0.0129, 0.0276,  ..., 0.0101, 0.0360, 0.0410],
        [0.0082, 0.0065, 0.0230,  ..., 0.0321, 0.0076, 0.0337],
        ...,
        [0.0105, 0.0314, 0.0173,  ..., 0.0093, 0.0437, 0.0155],
        [0.0151, 0.0460, 0.0477,  ..., 0.0282, 0.0378, 0.0225],
        [0.0262, 0.0413, 0.0499,  ..., 0.0142, 0.0124, 0.0215]])


## 7. DEFINING VALUES FOR THE RNN

In [23]:
# Number of epochs
num_epochs = 20
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

## 8. SETTING THE TRAIN SET

In [24]:
# Train data
train_set = Loader()

In [25]:
# Setting the DataLoader
# DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
#            batch_sampler=None, num_workers=0, collate_fn=None,
#            pin_memory=False, drop_last=False, timeout=0,
#            worker_init_fn=None, *, prefetch_factor=2,
#            persistent_workers=False)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)

## 9. TRAINING THE MODEL

In [26]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
        if cuda:
            x, y = x.cuda(), y.cuda()
            #print(x)
            #print(y)
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn( outputs.squeeze().float() , y.float() )
            losses.append( loss.item() )
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-26-45796941a992>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/20 [00:00<?, ?it/s]

iter #0 Loss: 10.623854272505817
iter #1 Loss: 3.2278581659507264
iter #2 Loss: 1.4946162930077604
iter #3 Loss: 1.1303561031056182
iter #4 Loss: 0.9957474983866562
iter #5 Loss: 0.9361988950873275
iter #6 Loss: 0.9071322955438853
iter #7 Loss: 0.8910524050902833
iter #8 Loss: 0.8823407218431878
iter #9 Loss: 0.8759878320462259
iter #10 Loss: 0.8721317475104271
iter #11 Loss: 0.8693393728007441
iter #12 Loss: 0.8673399230250922
iter #13 Loss: 0.8657844408088938
iter #14 Loss: 0.8642254796479364
iter #15 Loss: 0.8635577597581517
iter #16 Loss: 0.8625100359434972
iter #17 Loss: 0.8622411547414482
iter #18 Loss: 0.8616981686990889
iter #19 Loss: 0.8613700044277074


In [27]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
print( uw , "----" , iw )

user_factors.weight tensor([[0.6808, 0.6514, 0.6755,  ..., 0.6784, 0.6765, 0.6927],
        [0.6774, 0.6827, 0.6958,  ..., 0.7183, 0.7342, 0.6727],
        [0.6846, 0.6956, 0.7204,  ..., 0.6998, 0.7087, 0.7109],
        ...,
        [0.5429, 0.5588, 0.5459,  ..., 0.5451, 0.5523, 0.5738],
        [0.6325, 0.6385, 0.6306,  ..., 0.6340, 0.6293, 0.6111],
        [0.7642, 0.7331, 0.7605,  ..., 0.7379, 0.7354, 0.7512]],
       device='cuda:0')
item_factors.weight tensor([[0.7266, 0.7523, 0.7017,  ..., 0.7355, 0.7531, 0.7061],
        [0.7753, 0.7852, 0.7751,  ..., 0.7434, 0.7716, 0.7941],
        [0.3873, 0.4100, 0.4208,  ..., 0.4162, 0.3917, 0.4068],
        ...,
        [0.4501, 0.4712, 0.4589,  ..., 0.4485, 0.4842, 0.4576],
        [0.4538, 0.4849, 0.4886,  ..., 0.4665, 0.4775, 0.4637],
        [0.4611, 0.4764, 0.4870,  ..., 0.4487, 0.4484, 0.4590]],
       device='cuda:0')
tensor([[0.6808, 0.6514, 0.6755,  ..., 0.6784, 0.6765, 0.6927],
        [0.6774, 0.6827, 0.6958,  ..., 0.7183, 0.734

In [28]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [29]:
len(trained_movie_embeddings) # unique movie factor weights

1682

In [30]:
trained_movie_embeddings.shape

(1682, 8)

## 10. CLUSTERING BY KMEANS

In [31]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [32]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = df_rating.loc[ df_rating['item_id'] == movid ].count()[0]
    movs.append((movie_names[ movid ], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:20]:
    print("\t", mov)

Cluster #0
	 ('Liar Liar (1997)', 485)
	 ('Saint, The (1997)', 316)
	 ('Twister (1996)', 293)
	 ('Broken Arrow (1996)', 254)
	 ("Devil's Own, The (1997)", 240)
	 ('Murder at 1600 (1997)', 218)
	 ('Starship Troopers (1997)', 211)
	 ('Eraser (1996)', 206)
	 ('Tin Cup (1996)', 193)
	 ('Star Trek III: The Search for Spock (1984)', 171)
	 ('Mother (1996)', 169)
	 ('Everyone Says I Love You (1996)', 168)
	 ('Kingpin (1996)', 162)
	 ('First Wives Club, The (1996)', 160)
	 ('Dragonheart (1996)', 158)
	 ('Die Hard: With a Vengeance (1995)', 151)
	 ('Grumpier Old Men (1995)', 148)
	 ('River Wild, The (1994)', 146)
	 ("Jackie Chan's First Strike (1996)", 145)
	 ('Interview with the Vampire (1994)', 137)
Cluster #1
	 ('Spawn (1997)', 143)
	 ('Jungle2Jungle (1997)', 132)
	 ('Crash (1996)', 128)
	 ('Event Horizon (1997)', 127)
	 ('Batman Forever (1995)', 114)
	 ('Down Periscope (1996)', 101)
	 ('Escape from L.A. (1996)', 91)
	 ('Beautician and the Beast, The (1997)', 86)
	 ('Phantom, The (1996)', 80